# LSTM Recurrent Neural Network

Tutorial/example [here](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/)

## compatibility: tensorflow 2.0.0

In [134]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [135]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [136]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1802001489636942523
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14750996256471918034
physical_device_desc: "device: XLA_CPU device"
]


In [137]:
tf.test.is_gpu_available()

False

In [138]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [139]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [140]:
print('#unique chars: \t', len(chars))
print('Total chars: \t', len(raw_text))

#unique chars: 	 43
Total chars: 	 144342


In [141]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
n_chars = len(raw_text)
n_vocab = len(chars)

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

print ("Total Patterns: ", n_patterns)

Total Patterns:  144242


In [142]:
print('Example dataX (len =', len(dataX[1]), '):\n\n', dataX[0], '\n')

int_to_char = dict((i, c) for i, c in enumerate(chars))
seq_in = [int_to_char[value] for value in dataX[0]]

print(seq_in)

Example dataX (len = 100 ):

 [19, 24, 17, 32, 36, 21, 34, 1, 25, 10, 1, 20, 31, 39, 30, 1, 36, 24, 21, 1, 34, 17, 18, 18, 25, 36, 9, 24, 31, 28, 21, 0, 0, 17, 28, 25, 19, 21, 1, 39, 17, 35, 1, 18, 21, 23, 25, 30, 30, 25, 30, 23, 1, 36, 31, 1, 23, 21, 36, 1, 38, 21, 34, 41, 1, 36, 25, 34, 21, 20, 1, 31, 22, 1, 35, 25, 36, 36, 25, 30, 23, 1, 18, 41, 1, 24, 21, 34, 1, 35, 25, 35, 36, 21, 34, 1, 31, 30, 1, 36] 

['c', 'h', 'a', 'p', 't', 'e', 'r', ' ', 'i', '.', ' ', 'd', 'o', 'w', 'n', ' ', 't', 'h', 'e', ' ', 'r', 'a', 'b', 'b', 'i', 't', '-', 'h', 'o', 'l', 'e', '\n', '\n', 'a', 'l', 'i', 'c', 'e', ' ', 'w', 'a', 's', ' ', 'b', 'e', 'g', 'i', 'n', 'n', 'i', 'n', 'g', ' ', 't', 'o', ' ', 'g', 'e', 't', ' ', 'v', 'e', 'r', 'y', ' ', 't', 'i', 'r', 'e', 'd', ' ', 'o', 'f', ' ', 's', 'i', 't', 't', 'i', 'n', 'g', ' ', 'b', 'y', ' ', 'h', 'e', 'r', ' ', 's', 'i', 's', 't', 'e', 'r', ' ', 'o', 'n', ' ', 't']


In [143]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [144]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [145]:
# define the checkpoint
filepath="models\weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [146]:
# load the network weights
filename = "models\weights-improvement-20-1.8710.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [71]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Train on 144242 samples
Epoch 1/20
  1024/144242 [..............................] - ETA: 4:52 - loss: 2.0100

KeyboardInterrupt: 

In [147]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [148]:
import sys

In [178]:
inputString = "hello im a person and i like icecream and its super nice. and yummy"

x = inputString.rjust(100)

seed = [char_to_int[char] for char in x]


In [179]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = seed
print ("Seed: " + inputString)
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed: hello im a person and i like icecream and its super nice. and yummy
e a lort oi thar a coese of thit same whe harter was so toee to the thre whe had boong to the tooe of the gorse, and the tabbit sat to taed to the thrt oi the sabe to then to teee to the that would be io een iane and the tooe of the toieo of the table  bnd the world sat to toee to the that so tee that was to tee tha looe to tea tha had boong th the toid of the gorse, and the tabbit sas to taed to the thre whe had boong to the tooe of the gorse, and the tare the hor to an   she was to tee to the that hor hoes tite the haree hertens  bnd toe toint so the thant oa beries and the tooer of the tabli  bnd the tore of then so tee that saed to the thate tabdit  bnd the coost tat oi the tooe, and the horpt her iaad oo the coord oo the tas of the coort, and the tooe tht wo and the tooer of that so the thoe, 
'the soeer dirse ti the toict to the taid th the toiel tf the toier of the sabbit, and the toiel to the tooer of the